In [7]:
''' 
    Computation of transient moments using the block-matrix method in the bivariate setting.
    
    Note that we have Exponential Marks, meaning that the random jump sizes B_{ij} are Exponentially distributed (!)
'''



''' Initialize parameters '''

include_lambda = True # True/False to include lambda or not (to increase efficiency in simulation)
M = 10**3 # number of interval discretizations

fixed_params = [include_lambda, M]

''' Bivariate setting '''

d_bi = 2 # dimensionality of processes
l_inf_bi = [.5,.5] # base rates
B_bi = np.array([[1.5,.5],[.75,1.25]]) # Jump sizes. If random jump sizes B: then B equals expected value of parameters
alpha_bi = [3,2] # decay rate of exponential function
mu_bi = [1,2] # mean sojourn times of event arrivals

model_params_bi = [d_bi, l_inf_bi, B_bi, alpha_bi, mu_bi] 


''' Trivariate setting '''
d_tri = 3
l_inf_tri = [.3,1,.5]
B_tri = np.array([[.5, .3, .4],[.7, .5, .5],[.4, .2, .5]]) # jump sizes - one can allow random jump sizes B: replace B with expected value of B
alpha_tri = [2, 1.5, 2.5]
mu_tri = [1.5, .5 ,1]

model_params_tri = [d_tri, l_inf_tri, B_tri, alpha_tri, mu_tri]


# Check if stability condition is satisfied: spectral radius < 1

C_matrix = np.array([np.array(B_bi[i])/alpha_bi[i] for i in range(d_bi)])
print("Bivariate stability condition satified: ", max(abs(np.linalg.eigvals(C_matrix)))<1)
print("Value maximal eigenvalue: ", max(abs(np.linalg.eigvals(C_matrix))))

C_matrix = np.array([np.array(B_tri[i])/alpha_tri[i] for i in range(d_tri)])
print("Trivariate stability condition satified: ", max(abs(np.linalg.eigvals(C_matrix)))<1)
print("Value maximal eigenvalue: ", max(abs(np.linalg.eigvals(C_matrix))))

Bivariate stability condition satified:  True
Value maximal eigenvalue:  0.8201941016011038
Trivariate stability condition satified:  True
Value maximal eigenvalue:  0.6801897797540147


In [10]:
''' 
    Bivariate setting
    Block Matrix method
    Exponential marks (!)
'''


''' Lambda moments - first and second order '''

al1_bi = alpha_bi[0]*l_inf_bi[0]
al2_bi = alpha_bi[1]*l_inf_bi[1]
b = np.multiply(alpha_bi, l_inf_bi)
alphabar_1 = alpha_bi[0] - B_bi[0][0]
alphabar_2 = alpha_bi[1] - B_bi[1][1] 


''' Function to determine size of \lambda(t) moments stacked vector '''
def cursive_m(n):
    def m_n(k):
        return k+1
    return sum([m_n(k) for k in range(1,n+1)])


''' Q and lambda moments - first, second and third order '''

''' First order moments '''

# Define block matrices
M_01 = np.array([[-alphabar_1, B_bi[0][1]],[B_bi[1][0], -alphabar_2]])
M_10 = np.array([[-mu_bi[0], 0], [0, -mu_bi[1]]])
F_1_temp1 = np.concatenate((M_01, np.zeros((2,2))), axis=1)
F_1_temp2 = np.concatenate((np.identity(2),M_10), axis=1)
F_1 = np.concatenate((F_1_temp1, F_1_temp2))

joint_psi_1storder_0 = np.concatenate((l_inf_bi, np.zeros(cursive_m(1)))) 
joint_b_1 = np.concatenate((b, np.zeros(cursive_m(1))))

def exp_F1(t):
    return expm(F_1*t)

def Q_Lambda_stacked_1storder_RM(t):
    return np.dot(exp_F1(t), joint_psi_1storder_0) - np.dot(np.dot(np.linalg.inv(F_1),np.identity(4) - exp_F1(t)), joint_b_1)


''' Second order moments '''

# Define the block matrix
joint_psi_2ndorder_0 = np.concatenate((np.concatenate((joint_psi_1storder_0, np.array([l_inf_bi[0]**2, l_inf_bi[0]*l_inf_bi[1], l_inf_bi[1]**2]))), np.zeros(7)))
joint_b_2 = np.concatenate((b, np.zeros(12)))

M_02 = np.array([[-2*alphabar_1, 2*B_bi[0][1], 0],[B_bi[1][0], -alphabar_1-alphabar_2, B_bi[0][1]], [0, 2*B_bi[1][0], -2*alphabar_2]])
M_11 = np.array([[-alphabar_1 - mu_bi[0], B_bi[0][1], 0, 0],[B_bi[1][0], -alphabar_2 - mu_bi[0], 0, 0], [0, 0, -alphabar_1 - mu_bi[1], B_bi[0][1]],[0, 0, B_bi[1][0], -alphabar_2 - mu_bi[1]]])
M_20 = np.array([[-2*mu_bi[0], 0, 0], [0, - mu_bi[0] - mu_bi[1], 0], [0, 0, -2*mu_bi[1]]])
L_02 = np.array([[2*alpha_bi[0]*l_inf_bi[0] + 2*B_bi[0][0]**2, 2*B_bi[0][1]**2],[B_bi[0][0]*B_bi[1][0] + alpha_bi[1]*l_inf_bi[1], B_bi[1][1]*B_bi[0][1] + alpha_bi[0]*l_inf_bi[0]],[2*B_bi[1][0]**2, 2*alpha_bi[1]*l_inf_bi[1] + 2*B_bi[1][1]**2]])
K_11 = np.array([[1,0,0],[0,1,0],[0,1,0],[0,0,1]])
L_11_Q = np.array([[alpha_bi[0]*l_inf_bi[0], 0], [alpha_bi[1]*l_inf_bi[1], 0], [0, alpha_bi[0]*l_inf_bi[0]], [0, alpha_bi[1]*l_inf_bi[1]]])
L_11_lambda = np.array([[B_bi[0][0], 0],[B_bi[1][0], 0], [0, B_bi[0][1]], [0, B_bi[1][1]]])
K_20 = np.array([[2, 0, 0 ,0], [0, 1, 1, 0], [0, 0, 0, 2]])

F_2_temp1 = np.concatenate((F_1, np.zeros((4,10))),axis=1)
F_2_temp2 = np.concatenate((L_02, np.zeros((3,2)), M_02, np.zeros((3,7))), axis=1)
F_2_temp3 = np.concatenate((L_11_lambda, L_11_Q, K_11, M_11, np.zeros((4,3))), axis=1)
F_2_temp4 = np.concatenate((np.zeros((3,7)), K_20, M_20), axis=1)
F_2 = np.concatenate((F_2_temp1, F_2_temp2, F_2_temp3, F_2_temp4))

def exp_F2(t):
    return expm(F_2*t)

def Q_Lambda_stacked_2ndorder_RM(t):
    return np.dot(exp_F2(t), joint_psi_2ndorder_0) - np.dot(np.dot(np.linalg.inv(F_2),np.identity(14) - exp_F2(t)), joint_b_2)



''' Third order moments '''


# Define M matrices 
M_03 = np.array([[-3*alphabar_1, 3*B_bi[0][1], 0, 0], [B_bi[1][0], -2*alphabar_1 -alphabar_2, 2*B_bi[0][1], 0], [0, 2*B_bi[1][0], -alphabar_1-2*alphabar_2, B_bi[0][1]], [0, 0, 3*B_bi[1][0], -3*alphabar_2]])
M_12 = np.array([[-2*alphabar_1 - mu_bi[0], 2*B_bi[0][1], 0, 0, 0, 0], [B_bi[1][0], -alphabar_1-alphabar_2-mu_bi[0], B_bi[0][1], 0, 0, 0], [0, 2*B_bi[1][0], -2*alphabar_2 -mu_bi[0], 0, 0, 0], [0, 0, 0, -2*alphabar_1-mu_bi[1], 2*B_bi[0][1], 0], [0, 0, 0, B_bi[1][0], -alphabar_1-alphabar_2-mu_bi[1], B_bi[0][1]], [0, 0, 0, 0, 2*B_bi[1][0], -2*alphabar_2-mu_bi[1]]])
M_21 = np.array([[-alphabar_1 - 2*mu_bi[0], B_bi[0][1], 0, 0, 0, 0], [B_bi[1][0], -alphabar_2 - 2*mu_bi[0], 0, 0, 0, 0], [0, 0, -alphabar_1 -mu_bi[0]-mu_bi[1], B_bi[0][1], 0, 0], [0, 0, B_bi[1][0], -alphabar_2 -mu_bi[0]-mu_bi[1], 0, 0], [0, 0, 0, 0, -alphabar_1 -2*mu_bi[1], B_bi[0][1]], [0, 0, 0, 0, B_bi[1][0], -alphabar_2 -2*mu_bi[1]]])
M_30 = np.array([[-3*mu_bi[0], 0, 0, 0], [0, -2*mu_bi[0]-mu_bi[1], 0, 0], [0, 0, -mu_bi[0] -2*mu_bi[1], 0], [0, 0, 0, -3*mu_bi[1]]])

# Define A and L matrices 
L_03_L1st = np.array([[3*B_bi[0][0]**3, 3*B_bi[0][1]**3], [2*B_bi[0][0]**2*B_bi[1][0], 2*B_bi[0][1]**2*B_bi[1][1]], [B_bi[0][0]*2*B_bi[1][0]**2, B_bi[0][1]*2*B_bi[1][1]**2], [3*B_bi[1][0]**3, 3*B_bi[1][1]**3]])
L_03_L2nd = np.array([[3*2*B_bi[0][0]**2 + 3*al1_bi, 3*2*B_bi[0][1]**2, 0], [2*B_bi[0][0]*B_bi[1][0] + al2_bi, 2*B_bi[0][1]*B_bi[1][1] + 2*B_bi[0][0]**2 +2*al1_bi, 2*B_bi[0][1]**2], [2*B_bi[1][0]**2, 2*B_bi[1][0]*B_bi[0][0] + 2*B_bi[1][1]**2 + 2*al2_bi, 2*B_bi[0][1]*B_bi[1][1]+al1_bi], [0, 3*2*B_bi[1][0]**2, 3*2*B_bi[1][1]**2 + 3*al2_bi]])
L_03 = np.concatenate((L_03_L1st, np.zeros((4,2)), L_03_L2nd, np.zeros((4,7))), axis=1)

L_12_L1st = np.array([[2*B_bi[0][0]**2, 0], [B_bi[0][0]*B_bi[1][0], 0], [2*B_bi[1][0]**2, 0], [0, 2*B_bi[0][1]**2], [0, B_bi[0][1]*B_bi[1][1]], [0, 2*B_bi[1][1]**2]])
L_12_Qst = np.zeros((6,2))
L_12_L2nd = np.array([[2*B_bi[0][0], 0, 0], [B_bi[1][0], B_bi[0][0], 0], [0, 2*B_bi[1][0], 0], [0, 2*B_bi[0][1], 0], [0, B_bi[1][1], B_bi[0][1]], [0, 0, 2*B_bi[1][1]]])
L_12_QL2nd_temp = np.array([[2*B_bi[0][0]**2 + 2*al1_bi, 2*B_bi[0][1]**2], [B_bi[0][0]*B_bi[1][0] + al2_bi, B_bi[0][1]*B_bi[1][1] + al1_bi], [2*B_bi[1][0]**2, 2*B_bi[1][1]**2 + 2*al2_bi]])
L_12_QL2nd = np.concatenate((np.concatenate((L_12_QL2nd_temp, np.zeros((3,2))), axis =1), np.concatenate((np.zeros((3,2)), L_12_QL2nd_temp), axis =1)))
L_12_Q2nd = np.zeros((6,3))
L_12 = np.concatenate((L_12_L1st, L_12_Qst, L_12_L2nd, L_12_QL2nd, L_12_Q2nd), axis=1)
F_12 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

L_21_QL2nd = np.array([[2*B_bi[0][0], 0, 0, 0], [2*B_bi[1][0], 0, 0, 0], [0, B_bi[0][1], B_bi[0][0], 0], [0, B_bi[1][1], B_bi[1][0], 0], [0, 0, 0, 2*B_bi[0][1]], [0, 0, 0, 2*B_bi[1][1]]])
L_21_Q2nd = np.array([[al1_bi, 0, 0], [al2_bi, 0, 0], [0, al1_bi, 0], [0, al2_bi,0], [0, 0, al1_bi], [0, 0, al2_bi] ])
L_21 = np.concatenate((np.zeros((6,7)), L_21_QL2nd, L_21_Q2nd), axis=1)
F_21 = np.array([[2, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 2]])

F_30 = np.array([[3, 0, 0, 0, 0, 0], [0, 1, 2, 0, 0, 0], [0, 0, 0, 2, 1, 0], [0, 0, 0, 0, 0, 3]])


# Construct A_3_QL matrix
F_3_temp1 = np.concatenate((F_2, np.zeros((14,20))), axis=1)
F_3_temp2 = np.concatenate((L_03, M_03, np.zeros((4,16))), axis=1)
F_3_temp3 = np.concatenate((L_12, F_12, M_12, np.zeros((6,10))), axis=1)
F_3_temp4 = np.concatenate((L_21, np.zeros((6,4)), F_21, M_21, np.zeros((6,4))),axis=1)
F_3_temp5 = np.concatenate((np.zeros((4,24)), F_30, M_30), axis=1)
F_3 = np.concatenate((F_3_temp1, F_3_temp2, F_3_temp3, F_3_temp4, F_3_temp5))

def exp_F3(t):
    return expm(F_3*t)

joint_psi_3rdorder_0 = np.concatenate((np.concatenate((joint_psi_2ndorder_0, np.array([l_inf_bi[0]**3, l_inf_bi[0]**2*l_inf_bi[1],l_inf_bi[0]*l_inf_bi[1]**2, l_inf_bi[1]**3]))), np.zeros(16)))
joint_b_3 = np.concatenate((b, np.zeros(32)))

def Q_Lambda_stacked_3rdorder_RM(t):
    return np.dot(exp_F3(t), joint_psi_3rdorder_0) - np.dot(np.dot(np.linalg.inv(F_3),np.identity(34) - exp_F3(t)), joint_b_3)

